In [38]:
# import modules

import pandas as pd
import numpy as np
import requests
import re
import time
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [2]:
data = pd.read_csv('Crawling url.csv')
data

,keyword,title,views,url
0,ryuichi sakamoto,Ryuichi Sakamoto- 'Merry Christmas Mr Lawrence',"9,490,277",https://www.youtube.com/watch?v=LGs_vGt0MY8
1,ryuichi sakamoto,Ryuichi Sakamoto-Energy Flow,"7,184,948",https://www.youtube.com/watch?v=btyhpyJTyXg
2,ryuichi sakamoto,ryuichi sakamoto - rain(live),"6,031,819",https://www.youtube.com/watch?v=8tKfYwc4zxA
3,joe hisaishi,Spirited Away (2001) - The Name of Life (Instr...,"34,634,160",https://www.youtube.com/watch?v=ImPM5IDIYPs
4,joe hisaishi,Howl's Moving Castle [OST - Theme Song],"28,570,500",https://www.youtube.com/watch?v=UwxatzcYf9Q
5,joe hisaishi,Joe Hisaishi Live - Summer ( from Kikujiro ),"9,569,659",https://www.youtube.com/watch?v=qEb4TG10jW8


In [22]:
# Youtube URL array

url_arr = data.url.values
url_arr

array(['https://www.youtube.com/watch?v=LGs_vGt0MY8',
       'https://www.youtube.com/watch?v=btyhpyJTyXg',
       'https://www.youtube.com/watch?v=8tKfYwc4zxA',
       'https://www.youtube.com/watch?v=ImPM5IDIYPs',
       'https://www.youtube.com/watch?v=UwxatzcYf9Q',
       'https://www.youtube.com/watch?v=qEb4TG10jW8'], dtype=object)

In [14]:
# selenium 속도 향상용

options = webdriver.ChromeOptions()
prefs = {'profile.default_content_setting_values': {'cookies': 2, 'images': 2, 'javascript': 2, 
                            'plugins': 2, 'popups': 2, 'geolocation': 2, 
                            'notifications': 2, 'auto_select_certificate': 2, 'fullscreen': 2, 
                            'mouselock': 2, 'mixed_script': 2, 'media_stream': 2, 
                            'media_stream_mic': 2, 'media_stream_camera': 2, 'protocol_handlers': 2, 
                            'ppapi_broker': 2, 'automatic_downloads': 2, 'midi_sysex': 2, 
                            'push_messaging': 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop': 2, 
                            'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement': 2, 
                            'durable_storage': 2}}
options.add_experimental_option('prefs', prefs)
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
# 크롬창을 띄우지 않는 옵션을 넣는다	
options.add_argument('headless')	
options.add_argument('disable-gpu')	

In [57]:
# code test

if True:
    keyword = 'test'
    f = open('comment_crawling_{}.txt'.format(keyword), 'w', encoding='utf-8')
    if True:
            driver = webdriver.Chrome('./chromedriver', options=options)
            driver.get('https://www.youtube.com/watch?v=LGs_vGt0MY8')
            body = driver.find_element_by_tag_name("body")

            print('start') 
            #(스크롤 내리기) 
            num_of_pagedowns = 7 
            datetime.today()
            
            while num_of_pagedowns: 
                body.send_keys(Keys.PAGE_DOWN) 
                time.sleep(2) 
                num_of_pagedowns -= 1 
                
                try: 
                    driver.find_element_by_xpath('//*[@id="sort-menu"]').click() 
                    #driver.find_element_by_xpath('//*[@id="menu"]/a[@tabindex="-1"]').click() 
                    driver.find_element_by_xpath('//*[@id="menu"]/a[2]/paper-item/paper-item-body/div[text()="최근 날짜순"]').click() 
                except Exception as e: pass 
                
            #반복 (스크롤 횟수 설정 필요) 
            num_of_pagedowns = 30
            
            while True: 
                body.send_keys(Keys.PAGE_DOWN) 
                time.sleep(2) 
                num_of_pagedowns -= 1 
                
                #body = result.find("body") 
                #last = body.find_all('div', attrs={'id':'continuations'}) 
                #if last not in body: 
                #    break 

                html = driver.page_source 
                result = BeautifulSoup(html,'html.parser') 
                #print(html) 
                body2 = result.find("body") 
                #print(body) 
                title = body2.find_all('yt-formatted-string', attrs={'class':'style-scope ytd-video-primary-info-renderer'}) 
                title1 = title[0].get_text() 
                title1 = re.sub(r'[/\:*?<>"|!]',' ',title1) 
                #print(title1) 
                thread = body2.find_all('ytd-comment-renderer', attrs={'class':'style-scope ytd-comment-thread-renderer'}) 

                last = body2.find('yt-formatted-string', attrs={'class':'count-text style-scope ytd-comments-header-renderer'}) 
                last = last.string 
                #print(last) 
                last_t = re.sub('[^\d]','',last) 
                last_t = int(last_t) 
                plus_c = 0 
                plus_cmt = body2.find_all('span',attrs={'id':'more-text'}) 

                for count in plus_cmt: 
                    count = count.get_text().strip() 
                    #print(count) 
                    if count == '답글 보기': 
                        plus_c += 1 
                    else: 
                        cmts = re.sub('[^\d]','',count) 
                        plus_c += int(cmts) 

                #print(plus_c) 
                #print(last_t) 

                cnt = 0
                for items in thread: 
                    #댓글 내용 
                    div = items.find_all('yt-formatted-string', attrs={'id':'content-text'}) 
                    #기간(시점) 
                    div2 = items.select('yt-formatted-string > a')[0].get_text() 

                    for lists in div: 
                        #print(lists) 
                        if lists != None: 
                            try: 
                                cmt = lists.string 
                                #textcmt = re.sub(r"[^\w^?!',.%$]",' ',cmt) 
                                f.write(cmt + '\n') 
                                cnt += 1
                            except TypeError as e: pass 
                        else: pass 
                    #cmtlist.append([textcmt, div2])
                
                num_c = (cnt +plus_c)
                print(num_c) 
                if num_c >= (last_t-10) or num_c>=3500: 
                    break 
                    
            print('-'*50) 
    
    f.close()

start
19
19
19
19
19
35
39
39
39
39
59
59
59
59
77
77
77
77
77
96
96
96
96
112
116
116
116
134
134
134
134
150
154
154
154
171
174
174
174
185
193
193
193
193
202
213
213
213
229
232
232
232
251
251
251
251
260
271
271
271
283
291
291
291
298
311
311


KeyboardInterrupt: 

In [61]:
# Youtube comment text crawling code
# 출처: https://blog.naver.com/rahyu/221421098018

for keyword in np.unique(data.keyword.values):
    keyword_url_arr = data[data.keyword == keyword].url.values
    keyword_view_arr = data[data.keyword == keyword].views.values
    
    for i, url in enumerate(keyword_url_arr):
        f = open('comment_crawling_{}_{}.txt'.format(keyword, keyword_view_arr[i]), 'w', encoding='utf-8')
        print('------------------{}-----------------'.format(keyword))
        
        try:
            driver = webdriver.Chrome('./chromedriver', options=options)
            driver.get(url)
            body = driver.find_element_by_tag_name("body")

            print('start') 
            #(스크롤 내리기) 
            num_of_pagedowns = 7 
            datetime.today()
            
            while num_of_pagedowns: 
                body.send_keys(Keys.PAGE_DOWN) 
                time.sleep(2) 
                num_of_pagedowns -= 1 
                
                try: 
                    driver.find_element_by_xpath('//*[@id="sort-menu"]').click() 
                    #driver.find_element_by_xpath('//*[@id="menu"]/a[@tabindex="-1"]').click() 
                    driver.find_element_by_xpath('//*[@id="menu"]/a[2]/paper-item/paper-item-body/div[text()="최근 날짜순"]').click() 
                except Exception as e: pass 
                
            #반복 (스크롤 횟수 설정 필요) 
            num_of_pagedowns = 30
            
            while True: 
                body.send_keys(Keys.PAGE_DOWN) 
                time.sleep(2) 
                num_of_pagedowns -= 1 
                
                #body = result.find("body") 
                #last = body.find_all('div', attrs={'id':'continuations'}) 
                #if last not in body: 
                #    break 
            
                html = driver.page_source 
                result = BeautifulSoup(html,'html.parser') 
                #print(html) 
                body2 = result.find("body") 
                #print(body) 
                title = body2.find_all('yt-formatted-string', attrs={'class':'style-scope ytd-video-primary-info-renderer'}) 
                title1 = title[0].get_text() 
                title1 = re.sub(r'[/\:*?<>"|!]',' ',title1) 
                #print(title1) 
                thread = body2.find_all('ytd-comment-renderer', attrs={'class':'style-scope ytd-comment-thread-renderer'}) 

                last = body2.find('yt-formatted-string', attrs={'class':'count-text style-scope ytd-comments-header-renderer'}) 
                last = last.string 
                #print(last) 
                last_t = re.sub('[^\d]','',last) 
                last_t = int(last_t) 
                plus_c = 0 
                plus_cmt = body2.find_all('span',attrs={'id':'more-text'}) 

                for count in plus_cmt: 
                    count = count.get_text().strip() 
                    #print(count) 
                    if count == '답글 보기': 
                        plus_c += 1 
                    else: 
                        cmts = re.sub('[^\d]','',count) 
                        plus_c += int(cmts) 

                #print(plus_c) 
                #print(last_t) 

                cnt = 0
                for items in thread: 
                    #댓글 내용 
                    div = items.find_all('yt-formatted-string', attrs={'id':'content-text'}) 
                    #기간(시점) 
                    div2 = items.select('yt-formatted-string > a')[0].get_text() 

                    for lists in div: 
                        #print(lists) 
                        if lists != None: 
                            try: 
                                cmt = lists.string 
                                #textcmt = re.sub(r"[^\w^?!',.%$]",' ',cmt) 
                                f.write(cmt + '\n') 
                                cnt += 1
                            except TypeError as e: pass 
                        else: pass 
                    #cmtlist.append([textcmt, div2])
                
                num_c = (cnt + plus_c)
                print(num_c) 
                if num_c >= (last_t-10) or num_c >= 1000:  # 1000개 제한
                    break 
                    
            print('-'*50)
            
        except Exception as e:
            print('error')
            
        f.close()    

------------------joe hisaishi-----------------
start
20
20
20
20
20
38
38
38
38
38
57
58
58
58
58
58
72
77
77
77
77
96
96
96
96
96
107
116
116
116
116
133
133
133
133
133
147
152
152
152
152
165
171
171
171
171
188
191
191
191
191
203
210
210
210
210
225
230
230
230
230
240
249
249
249
249
259
269
269
269
269
279
288
288
288
288
298
307
307
307
307
319
327
327
327
327
335
345
345
345
345
357
365
365
365
365
375
382
382
382
392
402
402
402
402
413
422
422
422
422
432
441
441
441
441
452
460
460
460
460
468
480
480
480
480
491
500
500
500
506
519
519
519
519
528
538
538
538
549
557
557
557
557
561
576
576
576
576
581
596
596
596
596
603
614
614
614
614
621
633
633
633
633
637
653
653
653
653
660
672
672
672
672
677
692
692
692
692
700
711
711
711
711
716
730
730
730
730
738
750
750
750
750
757
769
769
769
769
775
788
788
788
788
794
808
808
808
808
815
828
828
828
828
834
847
847
847
847
852
867
867
867
867
873
887
887
887
887
894
905
905
905
905
911
925
925
925
925
928
944
944
944
944


KeyboardInterrupt: 

In [62]:
if True:
    if True:
        f = open('comment_crawling_{}_{}.txt'.format('ryuichi sakamoto', '6,031,819'), 'w', encoding='utf-8')
        print('------------------{}-----------------'.format('ryuichi sakamoto'))
        
        try:
            driver = webdriver.Chrome('./chromedriver', options=options)
            driver.get('https://www.youtube.com/watch?v=8tKfYwc4zxA')
            body = driver.find_element_by_tag_name("body")

            print('start') 
            #(스크롤 내리기) 
            num_of_pagedowns = 7 
            datetime.today()
            
            while num_of_pagedowns: 
                body.send_keys(Keys.PAGE_DOWN) 
                time.sleep(2) 
                num_of_pagedowns -= 1 
                
                try: 
                    driver.find_element_by_xpath('//*[@id="sort-menu"]').click() 
                    #driver.find_element_by_xpath('//*[@id="menu"]/a[@tabindex="-1"]').click() 
                    driver.find_element_by_xpath('//*[@id="menu"]/a[2]/paper-item/paper-item-body/div[text()="최근 날짜순"]').click() 
                except Exception as e: pass 
                
            #반복 (스크롤 횟수 설정 필요) 
            num_of_pagedowns = 30
            
            while True: 
                body.send_keys(Keys.PAGE_DOWN) 
                time.sleep(2) 
                num_of_pagedowns -= 1 
                
                #body = result.find("body") 
                #last = body.find_all('div', attrs={'id':'continuations'}) 
                #if last not in body: 
                #    break 
            
                html = driver.page_source 
                result = BeautifulSoup(html,'html.parser') 
                #print(html) 
                body2 = result.find("body") 
                #print(body) 
                title = body2.find_all('yt-formatted-string', attrs={'class':'style-scope ytd-video-primary-info-renderer'}) 
                title1 = title[0].get_text() 
                title1 = re.sub(r'[/\:*?<>"|!]',' ',title1) 
                #print(title1) 
                thread = body2.find_all('ytd-comment-renderer', attrs={'class':'style-scope ytd-comment-thread-renderer'}) 

                last = body2.find('yt-formatted-string', attrs={'class':'count-text style-scope ytd-comments-header-renderer'}) 
                last = last.string 
                #print(last) 
                last_t = re.sub('[^\d]','',last) 
                last_t = int(last_t) 
                plus_c = 0 
                plus_cmt = body2.find_all('span',attrs={'id':'more-text'}) 

                for count in plus_cmt: 
                    count = count.get_text().strip() 
                    #print(count) 
                    if count == '답글 보기': 
                        plus_c += 1 
                    else: 
                        cmts = re.sub('[^\d]','',count) 
                        plus_c += int(cmts) 

                #print(plus_c) 
                #print(last_t) 

                cnt = 0
                for items in thread: 
                    #댓글 내용 
                    div = items.find_all('yt-formatted-string', attrs={'id':'content-text'}) 
                    #기간(시점) 
                    div2 = items.select('yt-formatted-string > a')[0].get_text() 

                    for lists in div: 
                        #print(lists) 
                        if lists != None: 
                            try: 
                                cmt = lists.string 
                                #textcmt = re.sub(r"[^\w^?!',.%$]",' ',cmt) 
                                f.write(cmt + '\n') 
                                cnt += 1
                            except TypeError as e: pass 
                        else: pass 
                    #cmtlist.append([textcmt, div2])
                
                num_c = (cnt + plus_c)
                print(num_c) 
                if num_c >= (last_t-10) or num_c >= 1000:  # 1000개 제한
                    break 
                    
            print('-'*50)
            
        except Exception as e:
            print('error')
            
        f.close()    

------------------ryuichi sakamoto-----------------
start
20
20
20
20
38
38
38
38
38
58
58
58
58
78
78
78
78
78
98
98
98
98
117
117
117
117
136
136
136
136
136
156
156
156
156
176
176
176
176
196
196
196
196
216
216
216
216
236
236
236
236
256
256
256
256
276
276
276
276
276
296
296
296
296
316
316
316
316
335
335
335
335
355
355
355
355
375
375
375
375
394
394
394
394
413
413
413
413
433
433
433
433
453
453
453
453
473
473
473
473
493
493
493
493
513
513
513
513
532
532
532
532
552
552
552
552
572
572
572
572
591
591
591
591
611
611
611
611
631
631
631
631
651
651
651
651
671
671
671
671
691
691
691
691
711
711
711
711
731
731
731
731
751
751
751
751
771
771
771
771
791
791
791
791
811
811
811
811
831
831
831
831
851
851
851
851
871
871
871
871
891
891
891
891
911
911
911
911
911
931
931
931
931
931
951
951
951
951
951
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966
966


KeyboardInterrupt: 